# Introduction/Business Problem

# Topic - Mumbai Buyers and Tenants Choice Made Simple

Mumbai, the financial capital of India and the second most populous city in the country.When it comes to choosing a location to buy or take a house on rent,it becomes a difficult choice in Mumbai.

The buyers/tenants face difficulty in knowing the neighborhood well without visting the place. With Mumbai being known for its huge length and breadth, the report should be able to give them a snapshot of the areas they are looking out for.

The data analysis should make it simpler for the prospective buyers and tenants,to help them identify their area of choice.The report will help them in analyzing each area as to what services will be available close by.The report should list the skyscrapers in the particular area and hence the users get to know the name of the buildings,location and the prospective service providers in the neighbourhood.

In [29]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#import folium
# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium

# Data Section

The data will be extracted from the below wikipedia page where it lists the tallest buildings in Mumbai.

List of tallest buildings in Mumbai - Wikipedia The data will be scraped from the wikipedia page.The data will be cleaned where the rank,image and building type columns will not be used.

The latitude and longitude will be found for each location using geocoder API.
The venues will be explored based on the location's latitude and longitude using the FourSquare API.
The most common venues will be reported for a particular location which will give the user a clear picture about the most common service providers corresponding to each building and area.

In [30]:
url = 'https://en.wikipedia.org/wiki/List_of_tallest_buildings_in_Mumbai'
html1 = urlopen(url) 
soup = BeautifulSoup(html1, 'html.parser')

In [31]:
from IPython.display import display, HTML
def display_print(df2):
    return display( HTML( df2.to_html().replace("\\n","<br>") ) )


In [32]:
#Create array to hold the data we extract
table = soup.find_all('table',attrs={'class': 'wikitable sortable'})
pd.set_option('display.max_columns', None)
df_ready = pd.read_html(str(table))[0]
df_ready.rename(columns= { 'Building type': 'Buildingtype' }, inplace= True)
df_ready = df_ready[df_ready.Buildingtype == 'Residential']
df_ready = df_ready.drop(df_ready.columns[[0, 3, 7]], axis=1)
df_ready.reset_index(drop=True, inplace=True)
print(display_print(df_ready))
print("Data shape",df_ready.shape)

,Name,Location,Height,Floors,Year
0,Palais Royale,Worli,"320 metres (1,050 ft)",88,2020
1,World One,Lower Parel,285 metres (935 ft),76,2020[3][4][5]
2,World View,Lower Parel,285 metres (935 ft),76,2020[3][4][5]
3,Lodha Trump Tower,Lower Parel,279 metres (915 ft),76,2020[6]
4,Lodha Marquise,Lower Parel,279 metres (915 ft),76,2020[7]
5,Lodha Parkside,Lower Parel,279 metres (915 ft),76,2020[7]
6,Lodha Allura,Lower Parel,279 metres (915 ft),76,2020[7]
7,Lodha Kiara,Lower Parel,279 metres (915 ft),76,2020[7]
8,Omkar 1973 Tower A,Worli,268 metres (879 ft),73,2020[8]
9,Omkar 1973 Tower B,Worli,268 metres (879 ft),73,2020[8]


None
Data shape (139, 5)


In [33]:
from geopy.geocoders import Nominatim
latitude = list()
longitude = list()
address =""
address =""
geolocator = Nominatim(user_agent="mumbai_explorer")
i=0
while i < len(df_ready) :
   # print(df_ready['Location'][i])
    address = df_ready['Location'][i]
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    #print(latitude,longitude)
    i=i+1
df_ready["Latitude"]=np.array(latitude)
df_ready["Longitude"]=np.array(longitude)
print(display_print(df_ready))
print("Data shape",df_ready.shape)

,Name,Location,Height,Floors,Year,Latitude,Longitude
0,Palais Royale,Worli,"320 metres (1,050 ft)",88,2020,19.011696,72.818070
1,World One,Lower Parel,285 metres (935 ft),76,2020[3][4][5],18.996332,72.830860
2,World View,Lower Parel,285 metres (935 ft),76,2020[3][4][5],18.996332,72.830860
3,Lodha Trump Tower,Lower Parel,279 metres (915 ft),76,2020[6],18.996332,72.830860
4,Lodha Marquise,Lower Parel,279 metres (915 ft),76,2020[7],18.996332,72.830860
5,Lodha Parkside,Lower Parel,279 metres (915 ft),76,2020[7],18.996332,72.830860
6,Lodha Allura,Lower Parel,279 metres (915 ft),76,2020[7],18.996332,72.830860
7,Lodha Kiara,Lower Parel,279 metres (915 ft),76,2020[7],18.996332,72.830860
8,Omkar 1973 Tower A,Worli,268 metres (879 ft),73,2020[8],19.011696,72.818070
9,Omkar 1973 Tower B,Worli,268 metres (879 ft),73,2020[8],19.011696,72.818070


None
Data shape (139, 7)


In [34]:
df_uc = pd.read_html(str(table))[1]
df_uc = df_uc.drop(df_uc.columns[[0]], axis=1)
df_uc.rename(columns= { 'Expected year of completion': 'Year' }, inplace= True)
print(display_print(df_uc))
print("Data shape",df_uc.shape)

,Name,Location,Height,Floors,Year
0,MSRTC Maha Sentral T 1,Mumbai Central,"400 metres (1,312 ft)",98,2022
1,Three Sixty West,Worli,"361 metres (1,184 ft)",90,0[9]
2,Piramal Mahalaxami South Tower,Mahalaxmi,"320 metres (1,050 ft)",87,2022
3,Piramal Mahalaxami Central Tower,Mahalaxmi,"320 metres (1,050 ft)",87,2023
4,Piramal Mahalaxami North Tower,Mahalaxmi,"320 metres (1,050 ft)",87,2023
5,Lokhandwala Minerva,Mahalaxmi,"320 metres (1,050 ft)",83,2019[35][36]
6,Panchshil Ocean Towers 1,Marine Drive,"320 metres (1,050 ft)",75,2022 [37]
7,Panchshil Ocean Towers 2,Marine Drive,300 metres (984 ft),73,2022 [37]
8,Namaste Tower,Worli,"320 metres (1,050 ft)",62,2020[38]
9,Radius Harbour Heights A,Byculla,304 metres (997 ft),74,2021


None
Data shape (197, 5)


In [35]:
latitude = list()
longitude = list()
address =""
geolocator = Nominatim(user_agent="mumbai_explorer")
i=0
while i < len(df_uc) :
    #str_location = complete_data['Name'][i] +","+complete_data['Location'][i]
    address = df_uc['Location'][i]
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    #print(latitude,longitude)
    i=i+1
df_uc["Latitude"]=np.array(latitude)
df_uc["Longitude"]=np.array(longitude)
print(display_print(df_uc))
print("Data shape",df_uc.shape)

,Name,Location,Height,Floors,Year,Latitude,Longitude
0,MSRTC Maha Sentral T 1,Mumbai Central,"400 metres (1,312 ft)",98,2022,18.969586,72.819315
1,Three Sixty West,Worli,"361 metres (1,184 ft)",90,0[9],19.011696,72.818070
2,Piramal Mahalaxami South Tower,Mahalaxmi,"320 metres (1,050 ft)",87,2022,27.632457,85.379321
3,Piramal Mahalaxami Central Tower,Mahalaxmi,"320 metres (1,050 ft)",87,2023,27.632457,85.379321
4,Piramal Mahalaxami North Tower,Mahalaxmi,"320 metres (1,050 ft)",87,2023,27.632457,85.379321
5,Lokhandwala Minerva,Mahalaxmi,"320 metres (1,050 ft)",83,2019[35][36],27.632457,85.379321
6,Panchshil Ocean Towers 1,Marine Drive,"320 metres (1,050 ft)",75,2022 [37],18.941487,72.823846
7,Panchshil Ocean Towers 2,Marine Drive,300 metres (984 ft),73,2022 [37],18.941487,72.823846
8,Namaste Tower,Worli,"320 metres (1,050 ft)",62,2020[38],19.011696,72.818070
9,Radius Harbour Heights A,Byculla,304 metres (997 ft),74,2021,18.976622,72.832794


None
Data shape (197, 7)


In [36]:
complete_data = df_ready.append(df_uc,ignore_index=True)  
print(display_print(complete_data.head()))

,Name,Location,Height,Floors,Year,Latitude,Longitude
0,Palais Royale,Worli,"320 metres (1,050 ft)",88,2020,19.011696,72.81807
1,World One,Lower Parel,285 metres (935 ft),76,2020[3][4][5],18.996332,72.83086
2,World View,Lower Parel,285 metres (935 ft),76,2020[3][4][5],18.996332,72.83086
3,Lodha Trump Tower,Lower Parel,279 metres (915 ft),76,2020[6],18.996332,72.83086
4,Lodha Marquise,Lower Parel,279 metres (915 ft),76,2020[7],18.996332,72.83086


None


In [37]:
CLIENT_ID = 'WPHQ4ZFRPXYO2TFLNZRM2BQ1WLTXT40FAQKZ4BYO1USMQNUI' # your Foursquare ID
CLIENT_SECRET = 'CFJYBM04XN02NANX14R5XZKNKFUJLPXIG2S23BG4ZKLISRWG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WPHQ4ZFRPXYO2TFLNZRM2BQ1WLTXT40FAQKZ4BYO1USMQNUI
CLIENT_SECRET:CFJYBM04XN02NANX14R5XZKNKFUJLPXIG2S23BG4ZKLISRWG


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
mumbai_venues = getNearbyVenues(names=df_ready['Location'],
                                   latitudes=df_ready['Latitude'],
                                   longitudes=df_ready['Longitude']
                                  )
mumbai_venues
mumbai_venues.groupby('Neighborhood').count()

Worli
Lower Parel
Lower Parel
Lower Parel
Lower Parel
Lower Parel
Lower Parel
Lower Parel
Worli
Worli
Worli
Tardeo
Tardeo
Parel
Prabhadevi
Sewri
Worli
Altamount Road
Malad
Wadala
Sewri
Sewri
Sewri
Lower Parel
Mahalaxmi
Mahalaxmi
Sewri
Parel
Parel
Worli
Lower Parel
Sewri
Mahalaxmi
Kandivali
Kandivali
Mahalaxmi
Sewri
Byculla
Wadala
Parel
Lower Parel
Malad
Malad
Mahalaxmi
Mahalaxmi
Mahalaxmi
wadala
Parel
Malad
Parel
Mumbai Central
Mumbai Central
Mumbai Central
Goregaon
Goregaon
Goregaon
Worli
borivali
Kharghar
Mahalaxmi
worli
Altamount Road
Worli
Parel
Goregaon
Goregaon
Borivali
Dadar
Dadar
Goregaon
Goregaon
Goregaon
Wadala
Wadala
Wadala
Wadala
Wadala
Borivali
Mulund
Mulund
Wadala
Goregaon
Goregaon
Goregaon
Goregaon
Wadala
Mumbai
Mulund
Worli
Mulund
Mulund
Mulund
Mulund
Mulund
Mulund
Goregaon
Goregaon
Malad
Malad
Nana Chowk
Kanjurmarg
Kanjurmarg
Girgaum
Lalbaug
Lalbaug
Goregaon
Nana Chowk
Prabhadevi
Prabhadevi
Prabhadevi
Prabhadevi
Prabhadevi
Tardeo
Lokhandwala
Lokhandwala
Lokhandwala
Gor

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Altamount Road,8,8,8,8,8,8
Borivali,28,28,28,28,28,28
Byculla,12,12,12,12,12,12
Dadar,60,60,60,60,60,60
Girgaum,33,33,33,33,33,33
Goregaon,320,320,320,320,320,320
Kandivali,48,48,48,48,48,48
Kanjurmarg,48,48,48,48,48,48
Kharghar,2,2,2,2,2,2


Analyze each Neighbourhood

In [40]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head(10)
mumbai_onehot.shape
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped.shape

(26, 98)

In [41]:
num_top_venues = 10

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altamount Road----
                           venue  freq
0                  Grocery Store  0.50
1                  Auto Workshop  0.25
2                          Hotel  0.25
3                            ATM  0.00
4              Mobile Phone Shop  0.00
5                    Pizza Place  0.00
6                       Pharmacy  0.00
7                           Park  0.00
8  Paper / Office Supplies Store  0.00
9                    Opera House  0.00


----Borivali----
                venue  freq
0      Clothing Store  0.14
1  Chinese Restaurant  0.14
2      Ice Cream Shop  0.14
3        Burger Joint  0.07
4    Department Store  0.07
5                Café  0.07
6             Dog Run  0.07
7          Restaurant  0.07
8          Food Truck  0.07
9         Pizza Place  0.07


----Byculla----
               venue  freq
0  Indian Restaurant  0.25
1        Fish Market  0.08
2         Restaurant  0.08
3           Pharmacy  0.08
4               Park  0.08
5   Asian Restaurant  0.08
6     History 

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

print(display_print(neighborhoods_venues_sorted))

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altamount Road,Grocery Store,Hotel,Auto Workshop,Furniture / Home Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop
1,Borivali,Ice Cream Shop,Clothing Store,Chinese Restaurant,Snack Place,Dog Run,Pizza Place,Department Store,Restaurant,Café,Burger Joint
2,Byculla,Indian Restaurant,Restaurant,Asian Restaurant,Bakery,Bar,Fish Market,History Museum,Park,Pharmacy,Zoo
3,Dadar,Indian Restaurant,Fast Food Restaurant,Movie Theater,Coffee Shop,Café,Shopping Mall,Mobile Phone Shop,Flower Shop,Plaza,Maharashtrian Restaurant
4,Girgaum,Indian Restaurant,Ice Cream Shop,Café,Restaurant,Indie Movie Theater,Pizza Place,Coffee Shop,Gym,Juice Bar,Mexican Restaurant
5,Goregaon,Indian Restaurant,Fast Food Restaurant,Bookstore,Pizza Place,Restaurant,Bus Station,Seafood Restaurant,Snack Place,Design Studio,Bar
6,Kandivali,Train Station,Indian Restaurant,Snack Place,Fast Food Restaurant,Park,Diner,Dessert Shop,Bus Station,Breakfast Spot,Platform
7,Kanjurmarg,Train Station,Asian Restaurant,Multiplex,Coffee Shop,Café,Zoo,Food Truck,Farmers Market,Fast Food Restaurant,Fish Market
8,Kharghar,Indian Restaurant,Theme Park,Furniture / Home Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop
9,Lalbaug,Indian Restaurant,Plaza,Gourmet Shop,Rest Area,Maharashtrian Restaurant,Food Court,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant


None


In [59]:
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 0], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mumbai_merged = df_ready

# merge manhattan_grouped with toronto_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Location')
#mumbai_merged = mumbai_merged.drop(mumbai_merged.columns[[5,6]], axis=1)
#mumbai_merged.reset_index(drop=True, inplace=True)
mumbai_merged.head() 

,Name,Location,Height,Floors,Year,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Palais Royale,Worli,"320 metres (1,050 ft)",88,2020,19.011696,72.81807,2.0,Scenic Lookout,Indian Restaurant,Garden,Paper / Office Supplies Store,Hotel,Gym,Grocery Store,Electronics Store,Falafel Restaurant,Harbor / Marina
1,World One,Lower Parel,285 metres (935 ft),76,2020[3][4][5],18.996332,72.83086,0.0,Café,Indian Restaurant,Brewery,Indoor Play Area,Lounge,Mediterranean Restaurant,Fast Food Restaurant,Music Venue,Pub,Recreation Center
2,World View,Lower Parel,285 metres (935 ft),76,2020[3][4][5],18.996332,72.83086,0.0,Café,Indian Restaurant,Brewery,Indoor Play Area,Lounge,Mediterranean Restaurant,Fast Food Restaurant,Music Venue,Pub,Recreation Center
3,Lodha Trump Tower,Lower Parel,279 metres (915 ft),76,2020[6],18.996332,72.83086,0.0,Café,Indian Restaurant,Brewery,Indoor Play Area,Lounge,Mediterranean Restaurant,Fast Food Restaurant,Music Venue,Pub,Recreation Center
4,Lodha Marquise,Lower Parel,279 metres (915 ft),76,2020[7],18.996332,72.83086,0.0,Café,Indian Restaurant,Brewery,Indoor Play Area,Lounge,Mediterranean Restaurant,Fast Food Restaurant,Music Venue,Pub,Recreation Center


In [64]:
from geopy.geocoders import Nominatim
address = 'Mumbai,IN'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai City are {}, {}.'.format(latitude, longitude))
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Location'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='#fffff',#rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Mumbai City are 19.0759899, 72.8773928.


TypeError: list indices must be integers or slices, not float